<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [103]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
from sqlalchemy import text
import plotly.graph_objects as go
import plotly.express as px

In [104]:
with open('data/credentials.txt') as credents:
    dbname = credents.readline().strip()
    user = credents.readline().strip()
    password = credents.readline().strip()
    host = credents.readline().strip()
    port = int(credents.readline().strip())

In [105]:
connection = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}')

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [106]:
#Считаем количество строк в таблице вакансий 
query_3_1 = f'''
SELECT COUNT(*)
FROM vacancies
'''

In [107]:
vacancies_count = pd.read_sql_query(query_3_1, connection)
print(f'Всего вакансий в таблице vacancies: {vacancies_count.iloc[0, 0]}')

Всего вакансий в таблице vacancies: 49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [108]:
#Считаем количество строк в таблице работодателей
query_3_2 = f'''
SELECT COUNT(*)
FROM employers
'''

In [109]:
employers_count = pd.read_sql_query(query_3_2, connection)
print(f'Всего работодателей в таблице employers: {employers_count.iloc[0, 0]}')

Всего работодателей в таблице employers: 23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [110]:
#Считаем количество строк в таблице регионов
query_3_3 = f'''
SELECT COUNT(*)
FROM areas
'''

In [111]:
areas_count = pd.read_sql_query(query_3_3, connection)
print(f'Всего регионов в таблице areas: {areas_count.iloc[0, 0]}')

Всего регионов в таблице areas: 1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [112]:
#Считаем количество строк в таблице сфер деятельности
query_3_4 = f'''
SELECT COUNT(*)
FROM industries
'''

In [113]:
industries_count = pd.read_sql_query(query_3_4, connection)
print(f'Всего сфер деятельности в таблице industries: {industries_count.iloc[0, 0]}')

Всего сфер деятельности в таблице industries: 294


***

## Промежуточные выводы
* На каждого работодателя в среднем приходится по 2 вакансии (2,09). Скорее всего, у крупных компаний значительно больше вакансий, чем 2. Можно предположить, что некоторые работодатели просто не ищут сотрудников.
* Всего в базе данных представлено 1362 города. Судя по информации в Википедии, в России насчитывается 1118 городов. Возможно, в качестве региона указываются не только города. Или в базе также представлены зарубежные вакансии. Кроме того, есть вариант, что пользователи вручную указывали регион и могли по-разному обозначить один и тот же город (например "Москва" и "г. Москва").
* Сфер деятельности достаточно много (294), но не очень понятно, насколько они уникальны, т.е. есть ли сферы разные по названию, но очень похожие по содержанию. 

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [114]:
query_4_1 = f'''
SELECT
	a.name AS area,
	COUNT(*) AS cnt	
FROM areas AS a
LEFT JOIN vacancies AS v ON a.id = v.area_id
GROUP BY a.name
ORDER BY COUNT(*) DESC
'''

In [115]:
top_areas = pd.read_sql_query(query_4_1, connection)
print('Вот первые пять регионов по количеству вакансий:')
display(top_areas.head())

Вот первые пять регионов по количеству вакансий:


,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [116]:
query_4_2 = f'''
SELECT
	COUNT(*) AS not_null_sal
FROM vacancies
WHERE (salary_from IS NOT NULL) OR (salary_to IS NOT NULL)
'''

In [117]:
top_areas = pd.read_sql_query(query_4_2, connection)
print(f'Количество вакансий, где указано хотя бы одно из двух полей зарплаты: {top_areas.iloc[0, 0]}')


Количество вакансий, где указано хотя бы одно из двух полей зарплаты: 24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [118]:
query_4_3 = f'''
SELECT
	ROUND(AVG(salary_from)) AS sal_from_avg,
	ROUND(AVG(salary_to)) AS salary_to_avg
FROM vacancies
'''

In [119]:
avg_sal = pd.read_sql_query(query_4_3, connection)
print(f'Среднее значение нижней границы зарплатной вилки: {avg_sal.iloc[0, 0]}')
print(f'Среднее значение верхней границы зарплатной вилки: {avg_sal.iloc[0, 1]}')


Среднее значение нижней границы зарплатной вилки: 71065.0
Среднее значение верхней границы зарплатной вилки: 110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [120]:
query_4_4 = f'''
SELECT
	schedule,
	employment,
	COUNT(*) AS count
FROM vacancies
GROUP BY schedule, employment
ORDER BY COUNT(*) DESC
'''

In [121]:
pop_vac = pd.read_sql_query(query_4_4, connection)
print('Количество вакансий по типам рабочего графика и трудоустройства, упорядоченное по убыванию:')
display(pop_vac)

Количество вакансий по типам рабочего графика и трудоустройства, упорядоченное по убыванию:


,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [122]:
query_4_5 = f'''
SELECT
	experience,
	COUNT(*) AS count
FROM vacancies
GROUP BY experience
ORDER BY COUNT(*)
'''

In [123]:
exp_vac = pd.read_sql_query(query_4_5, connection)
print('Количество вакансий в зависимости от требуемого опыта работы, упорядоченное по возрастанию:')
display(exp_vac)

Количество вакансий в зависимости от требуемого опыта работы, упорядоченное по возрастанию:


,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

In [124]:
# Выполним дополнительные вычисления
# Узнаем, сколько вакансий кроме той, что предполагает полный день и полную занятость
print(f'Сумма количества всех вакансий в таблице из п.4 без первой строки: {pop_vac["count"][1:].sum()}')

Сумма количества всех вакансий в таблице из п.4 без первой строки: 13830


## Промежуточные выводы
* Неудивительно, что по количеству вакансий в России первые три места занимают Москва, Санкт-Петербург и Новосибирск, так как это три города России с самым большим населением. Также можно заметить, что наше предположение о наличии зарубежных вакансий подтвердилось. Третье и пятое места занимают столица Беларуси и бывшая столица Казахстана соответственно.
* Часто в вакансиях вообще не указывают предлагаемую зарплату: хотя бы одно из двух полей (минимальная или максимальная зарплата) указано только в 24073 вакансиях, то есть менее чем в 50% (24073 из 49197).
* В среднем предлагается зарплата от 71065 до 110537. Средние показатели в данном случае не очень информативны. Это довольно высокие цифры для регионов и средние (или даже низкие) для Москвы и Санкт-Петербурга.
* Чаще всего ищут сотрудников на полный день и полную занятость — 35367 вакансий. Это почти в 3 раза больше чем всех остальных вакансий (13830).
* Чаще всего ищут сотрудников с опытом работы от 1 до 3 лет — 26152. Это специалисты, которых не нужно готовить с нуля, они могут начать работать и приносить результат сразу. Реже всего ищут сотрудников с опытом более 6 лет — 1337. Обычно это узкие специалисты на руководящие должности, поэтому их много не нужно.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [125]:
query_5_1 = f'''
SELECT
	e.name,
	COUNT(*) AS count
FROM employers AS e
LEFT JOIN vacancies AS v ON e.id = v.employer_id
GROUP BY e.id
ORDER BY COUNT(*) DESC
LIMIT 5
'''

In [126]:
emp_vac = pd.read_sql_query(query_5_1, connection)
print('Первые пять работодателей по количеству вакансий:')
display(emp_vac)

Первые пять работодателей по количеству вакансий:


,name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [127]:
# Упорядочим итоговую таблицы по количеству вакансий (по возрастанию), а затем по количеству
# работодателей (по убыванию), чтобы ответить на поставленный вопрос.
query_5_2 = f'''
SELECT
	a.name,
	COUNT(e.id) AS emp_count,
	COUNT(v.id) AS vac_count
FROM areas AS a
LEFT JOIN vacancies AS v ON a.id = v.area_id
JOIN employers AS e ON e.area = a.id
GROUP BY a.name
ORDER BY 3, 2 DESC
'''

In [128]:
area_vac = pd.read_sql_query(query_5_2, connection)
print('Количество вакансий по работодателям и регионам:')
display(area_vac.head())

Количество вакансий по работодателям и регионам:


,name,emp_count,vac_count
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Ростовская область,18,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [129]:
query_5_3 = f'''
SELECT
	e.name,
	COUNT(DISTINCT v.area_id) AS vac_count
FROM employers AS e
JOIN vacancies AS v ON e.id = v.employer_id
GROUP BY e.name
ORDER BY 2 DESC
'''

In [130]:
emp_vac = pd.read_sql_query(query_5_3, connection)
print('Количество регионов, для которых опубликованы вакансии, по работодателям:')
display(emp_vac.head())

Количество регионов, для которых опубликованы вакансии, по работодателям:


,name,vac_count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [131]:
query_5_4 = f'''
SELECT
	COUNT(*)
FROM employers AS e
LEFT JOIN employers_industries AS ei ON e.id = ei.employer_id
WHERE ei.industry_id IS NULL
'''

In [132]:
emp_wo_ind = pd.read_sql_query(query_5_4, connection)
print(f'Количество работодателей, у которых не указана сфера деятельности: {emp_wo_ind.iloc[0, 0]}')

Количество работодателей, у которых не указана сфера деятельности: 8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [133]:
query_5_5 = f'''
SELECT
	e.name
FROM employers AS e
LEFT JOIN employers_industries AS ei ON e.id = ei.employer_id
GROUP BY e.name
HAVING COUNT(industry_id) = 4
ORDER BY 1
OFFSET 2
LIMIT 1
'''

In [134]:
specific_emp = pd.read_sql_query(query_5_5, connection)
print(f'Компания, третья в алфавитном списке компаний, у которых указано четыре сферы деятельности: {specific_emp.iloc[0, 0]}')

Компания, третья в алфавитном списке компаний, у которых указано четыре сферы деятельности: 2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [135]:
query_5_6 = f'''
SELECT
	COUNT(*)
FROM employers AS e
LEFT JOIN employers_industries AS ei ON e.id = ei.employer_id
JOIN industries AS i ON ei.industry_id = i.id
WHERE i.name = 'Разработка программного обеспечения'
'''

In [136]:
dev_count = pd.read_sql_query(query_5_6, connection)
print(f'Количества работодателей со сферой деятельности "Разработка программного обеспечения": {dev_count.iloc[0, 0]}')


Количества работодателей со сферой деятельности "Разработка программного обеспечения": 3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [157]:
'''
Найдем список городов-миллионников на сайте википедии
Список городов находится в таблице, поэтому ищем содержимое тегов tbody
Вручную перебираем все таблицы и выясняем, что нужные нам города в таблице с индексом 10
Так как города оформлены в виде ссылок ищем содержимое тегов a
Вручную выясняем, что нужные нам города находятся в тегах с индексами с 2 по 17
Создаем строку, которую можно передать в SQL-запрос
'''
with open('data/url.txt') as url_link:
      url = url_link.readline().strip()
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')
tables = page.find_all('tbody')
links = tables[10].find_all('a')
million_cities = ["'" + x.text + "'" for x in links[2:18]]
for_query = ", ".join(million_cities)

In [138]:
query_5_7 = f'''
SELECT
        a.name,
        COUNT(*)
FROM employers AS e
JOIN vacancies AS v ON e.id = v.employer_id
JOIN areas AS a ON a.id = v.area_id
WHERE e.name = 'Яндекс' AND a.name IN ({for_query})
GROUP BY a.name

UNION ALL

SELECT 
        'total',
        COUNT(*)
FROM employers AS e
JOIN vacancies AS v ON e.id = v.employer_id
JOIN areas AS a ON a.id = v.area_id
WHERE e.name = 'Яндекс' AND a.name IN ({for_query})
ORDER BY 2
'''

In [139]:
yand_vac = pd.read_sql_query(query_5_7, connection)
print('Количество вакансий компании "Яндекс" в городах-миллионниках:')
display(yand_vac)

Количество вакансий компании "Яндекс" в городах-миллионниках:


,name,count
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

## Промежуточные выводы
* Больше всего вакансий предлагает "Яндекс" — 1933, 485 из них — в городах-миллионниках. Также "Яндекс" на первом месте по количеству регионов, где они ищет сотрудников — 181.
* Всего в России не менее 410 работодателей.
* Приблизительно у 36% работодателей (8419 из 23501) не указана сфера деятельности, но есть компании, которые связаны не менее чем с четырьмя сферами деятельности.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [140]:
query_6_1 = text('''
SELECT
    COUNT(*)
FROM vacancies
WHERE (LOWER(name) LIKE '%data%') OR (LOWER(name) LIKE '%данн%')
''')

In [141]:
data_vac = pd.read_sql_query(query_6_1, connection)
print(f'Количество вакансий, имеющих отношение к данным: {data_vac.iloc[0, 0]}')

Количество вакансий, имеющих отношение к данным: 1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [142]:
query_6_2 = text('''
SELECT
      COUNT(*)
FROM vacancies
WHERE ((LOWER(name) LIKE '%data scientist%') OR (LOWER(name) LIKE '%data science%') OR
      (LOWER(name) LIKE '%исследователь данных%') OR (LOWER(name) LIKE '%machine learning%') OR
      (LOWER(name) LIKE '%машинн%обучен%') OR (LOWER(name) LIKE '%ml%')) AND (NOT LOWER(name) LIKE '%html%') AND
      ((LOWER(name) LIKE '%junior%') OR (experience = 'Нет опыта') OR (employment = 'Стажировка'))
''')

In [143]:
jun_data_vac = pd.read_sql_query(query_6_2, connection)
print(f'Количество вакансий для начинающих дата-сайентистов: {jun_data_vac.iloc[0, 0]}')

Количество вакансий для начинающих дата-сайентистов: 51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [144]:
query_6_3 = text('''
SELECT
    COUNT(*)
FROM vacancies
WHERE ((LOWER(name) LIKE '%data scientist%') OR (LOWER(name) LIKE '%data science%') OR
      (LOWER(name) LIKE '%исследователь данных%') OR (LOWER(name) LIKE '%machine learning%') OR
      (LOWER(name) LIKE '%машинн%обучен%') OR (name LIKE '%ML%')) AND (NOT name LIKE '%HTML%') AND
      ((key_skills LIKE '%SQL%') OR (key_skills LIKE '%postgres%'))
''')

In [145]:
sql_vac = pd.read_sql_query(query_6_3, connection)
print(f'Количество вакансий для DS с ключевым навыком SQL или postgres: {sql_vac.iloc[0, 0]}')

Количество вакансий для DS с ключевым навыком SQL или postgres: 201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [146]:
query_6_4 = text('''
SELECT
    COUNT(*)
FROM vacancies
WHERE ((LOWER(name) LIKE '%data scientist%') OR (LOWER(name) LIKE '%data science%') OR
      (LOWER(name) LIKE '%исследователь данных%') OR (LOWER(name) LIKE '%machine learning%') OR
      (LOWER(name) LIKE '%машинн%обучен%') OR (name LIKE '%ML%')) AND (NOT name LIKE '%HTML%') AND
      (LOWER(key_skills) LIKE '%python%')
''')

In [147]:
py_vac = pd.read_sql_query(query_6_4, connection)
print(f'Количество вакансий для DS с ключевым навыком Python: {py_vac.iloc[0, 0]}')

Количество вакансий для DS с ключевым навыком Python: 351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [148]:
#Запрос на получение столбца ключевых навыков, не учитываем незаполненные строки
query_6_5 = text('''
SELECT
    key_skills
FROM vacancies
WHERE ((LOWER(name) LIKE '%data scientist%') OR (LOWER(name) LIKE '%data science%') OR
      (LOWER(name) LIKE '%исследователь данных%') OR (LOWER(name) LIKE '%machine learning%') OR
      (LOWER(name) LIKE '%машинн%обучен%') OR (name LIKE '%ML%')) AND (NOT name LIKE '%HTML%') AND
      (key_skills IS NOT NULL)
''')

In [149]:
key_skills = pd.read_sql_query(query_6_5, connection)
#В таблице навыки разделены знаками табуляции(\t), разбиваем строки по ним
key_skills['key_skills'] = key_skills['key_skills'].str.split('\t')
#Считаем количество элементов(навыков) в получившемся списке и записываем в новый столбец
key_skills['skills_num'] = key_skills['key_skills'].apply(len)
#Считаем среднее по столбцу и получаем результат
result = key_skills['skills_num'].mean()
print(f'Среднее количество ключевых навыков в вакансиях для DS: {result:.2f}')

Среднее количество ключевых навыков в вакансиях для DS: 6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [150]:
#Создадим временную таблицу с зарплатой, вычисляемой согласно пункту 2. Используем следующее выражение:
#(CAST(COALESCE(salary_from + salary_to, 1) AS FLOAT) / COALESCE(2 * (salary_from + salary_to), 1))
#Оно принимает значение 1, если одно из полей зарплаты пустое, и 1/2 в противном случае
#Изначально значения имеют формат INT, поэтому переводим числитель в FLOAT, чтобы можно было получить 0.5
query_6_6 = text('''
WITH temp_table AS
(
SELECT
	experience,
	salary_from,
	salary_to,
	(COALESCE(salary_from, 0) + COALESCE(salary_to, 0)) *
	(CAST(COALESCE(salary_from + salary_to, 1) AS FLOAT) / COALESCE(2 * (salary_from + salary_to), 1)) AS salary
FROM vacancies
WHERE ((LOWER(name) LIKE '%data scientist%') OR (LOWER(name) LIKE '%data science%') OR
      (LOWER(name) LIKE '%исследователь данных%') OR (LOWER(name) LIKE '%machine learning%') OR
      (LOWER(name) LIKE '%машинн%обучен%') OR (name LIKE '%ML%')) AND (NOT name LIKE '%HTML%') AND
	  (salary_from IS NOT NULL OR salary_to IS NOT NULL)
)

SELECT
	experience,
	AVG(salary) AS avg_sal
FROM temp_table
GROUP BY experience
''')

In [151]:
avg_sal = pd.read_sql_query(query_6_6, connection)
print('Средняя зарплата в зависимости от опыта:')
avg_sal

Средняя зарплата в зависимости от опыта:


,experience,avg_sal
0,От 3 до 6 лет,243114.666667
1,От 1 года до 3 лет,139674.750000
2,Нет опыта,74642.857143


***

In [152]:
# Дополнительные запросы
# Найдем, сколько всего вакансий для DS с помощью следующего запроса
'''
SELECT
    COUNT(*)
FROM vacancies
WHERE ((LOWER(name) LIKE '%data scientist%') OR (LOWER(name) LIKE '%data science%') OR
      (LOWER(name) LIKE '%исследователь данных%') OR (LOWER(name) LIKE '%machine learning%') OR
      (LOWER(name) LIKE '%машинн%обучен%') OR (name LIKE '%ML%')) AND (NOT name LIKE '%HTML%')
'''
# Ответ: 480
# Найдем 10 самых востребованных навыков DS
print('10 самых востребованных навыков DS:')
display(key_skills['key_skills'].explode().value_counts().head(10))

10 самых востребованных навыков DS:


Python                       348
SQL                          191
Machine Learning             114
Git                           66
Математическая статистика     62
Data Analysis                 54
Data Science                  52
Pandas                        52
ML                            49
Английский язык               48
Name: key_skills, dtype: int64

# Промежуточные выводы
* Вакансий для дужниор дата-сайентистов чуть больше 10% (51 из 480).
* Средняя зарплата, предлагаемая дужниор дата-сайентистам чуть выше минимальной предлагаемой в среднем по рынку (74643 против 71065). Средняя зарплата специалиста DS с минимальным опытом значительно выше максимальной предлагаемой в среднем по рынку (139675 против 110537). Для специалистов с опытом более 6 лет зарплата вообще не указана. Возможно, последнее объясняется слишком большой возможной разницей в вилке.
* Знание Python требуется в 73% вакансий (351 из 480), а знание SQL или postgres — в 42% (201 из 480). Также среди самых популярных навыков Machine Learning, Git, Математическая статистика, Pandas и английский язык. Это как раз те навыки, которые в среднем специалистам DS нужно освоить (более 6 навыков, мы перечислили 7).

# Общий вывод по проекту

* В базе данных содержатся вакансии как российских, так и зарубежных компаний. Штат некоторых компаний полностью укомплектован и они не ищут специалистов.
* Проще всего устроится на работу в Москве, Санкт-Петербурге или Новосибирске на полный день и полную занятость кандидатам с опытом работы от 1 до 3 лет.
* В России не менее 410 работодателей. Самый активный из них — Яндекс. У этой компании 1933 вакансии в 281 регионе. У работодателей может быть несколько сфер деятельности.
* Вакансий, связанных с DS, чуть меньше 10% от общей массы (480 из 49197). Зарплата выше средней по рынку: 74643 у джуниоров, через 1–3 года работы она вырастает до 139675, что выше средней верхней границы по рынку.
* Чтобы устроиться на работу, специалисту DS в среднем нужно освоить 6–7 ключевых навыков. Самые популярные из них — Python, SQL и Machine Learning. 

In [153]:
# Давайте узнаем, в каких компаниях наибольший спрос на специалистов DS (топ-10)
additional_query_1 = text(
'''
SELECT
    e.name,
	COUNT(*)
FROM vacancies AS v
LEFT JOIN employers AS e ON e.id = v.employer_id
WHERE ((LOWER(v.name) LIKE '%data scientist%') OR (LOWER(v.name) LIKE '%data science%') OR
      (LOWER(v.name) LIKE '%исследователь данных%') OR (LOWER(v.name) LIKE '%machine learning%') OR
      (LOWER(v.name) LIKE '%машинн%обучен%') OR (v.name LIKE '%ML%')) AND (NOT v.name LIKE '%HTML%')
GROUP BY e.name
ORDER BY 2 DESC
LIMIT 10
'''
)
pop_companies = pd.read_sql_query(additional_query_1, connection)
fig = go.Figure()
fig.add_trace(go.Bar(x=pop_companies['name'], y=pop_companies['count']))
fig.update_layout(title='Топ-10 компаний по найму DS-специалистов')
fig.update_yaxes(title = 'Количество вакансий')
fig.show()

![Bar](data/image-1.png "DS vacancies by company")

In [154]:
# Дополнительное исследование
# Давайте узнаем, в каких городах наибольший спрос на специалистов DS (топ-10)
additional_query_2 = text(
'''
SELECT
    a.name,
	COUNT(*)
FROM vacancies AS v
LEFT JOIN areas AS a ON a.id = v.area_id
WHERE ((LOWER(v.name) LIKE '%data scientist%') OR (LOWER(v.name) LIKE '%data science%') OR
      (LOWER(v.name) LIKE '%исследователь данных%') OR (LOWER(v.name) LIKE '%machine learning%') OR
      (LOWER(v.name) LIKE '%машинн%обучен%') OR (v.name LIKE '%ML%')) AND (NOT v.name LIKE '%HTML%')
GROUP BY a.name
ORDER BY 2 DESC
LIMIT 10
'''
)
pop_cities = pd.read_sql_query(additional_query_2, connection)
fig = go.Figure()
fig.add_trace(go.Bar(x=pop_cities['name'], y=pop_cities['count']))
fig.update_layout(title='Топ-10 городов по найму DS-специалистов')
fig.update_yaxes(title = 'Количество вакансий')
fig.show()

![Bar](data/image-2.png "DS vacancies by city")

In [155]:
# Давайте узнаем, типы трудоустройства и рабочих графиков требуются от специалистов DS
additional_query_3 = text(
'''
SELECT
    schedule,
	employment,
	COUNT(*)
FROM vacancies
WHERE ((LOWER(name) LIKE '%data scientist%') OR (LOWER(name) LIKE '%data science%') OR
      (LOWER(name) LIKE '%исследователь данных%') OR (LOWER(name) LIKE '%machine learning%') OR
      (LOWER(name) LIKE '%машинн%обучен%') OR (name LIKE '%ML%')) AND (NOT name LIKE '%HTML%')
GROUP BY schedule, employment
ORDER BY 3 DESC
'''
)
pop_sch_emp = pd.read_sql_query(additional_query_3, connection)
pop_cities = pd.read_sql_query(additional_query_2, connection)
fig = px.bar(
    pop_sch_emp,
    x="schedule",
    y="count",
    color="employment",
    barmode="group",
    title="Типы трудоустройства и рабочих графиков для DS-специалистов",
    labels={"employment" : "Рабочий график"}
)
fig.update_xaxes(title = 'График')
fig.update_yaxes(title = 'Количество вакансий')
fig.show()

![Bar](data/image-3.png "DS vacancies by schedule and employment")

## Дополнительные выводы
* Если претендент хочет повысить свои шансы на получение работы в сфере DS, ему нужно обратить внимание на требования компаний, которые чаще ищут таких специалистов. Например, неплохо бы изучить стек технологий, которые требуются для работы в СБЕРе (у этой компании 37 вакансий для специалистов DS).
* Распределение по количеству вакансий в городах похоже на распределение по всем вакансиям — первые три города: Москва, Санкт-Петербург и Новосибирск.
* Так же кандидату стоит задуматься о переезде в один из этих городов, так как большинство вакансий предполагают полный день с полной занятостью. Таких вакансий — примерно 66% (319 из 480).
